# <img style="float: right;"  src="images/jp.png" width="200">

# DC Motor

Description of a DC Motor operation

A DC motor can be described by the block diagram shown below

![Motor blocks](images/MotorBlocks.png)

The input motor voltage $V$ is converted to current $I$ by the motor winding resistance $R$. The current gives the motor torque $Tm$ thanks to the motor constant $k$.

The effective torque on the motor axis is obtained by substractiong the external load torque $Tl$ and the friction torque $Tf$ from the motor torque.

The effective torque is converted to **angular acceleration** **$\alpha$** depending on the inertial momentum $J$. Time integration of the angular acceleration **$\alpha$** gives the **angular speed $\omega$** and time integration of the angular speed gives the **axis angle $\theta$**.

The torque friction $Tf$ can be considered proportional to the angular speed.

The model also includes the electromagnetic force **EMF** generated by the motor movement that reduces the effective voltage on the motor.

Motor equations are, thus:

$$T=\frac{k}{R}(V-k \cdot \omega) - \mu \cdot \omega -T_{L}$$

$$\omega=\frac{1}{J}\int T \cdot dt$$

$$\theta=\int \omega \cdot dt$$

## Loading Python modules

We will simulate the motor operation. But, before that, we need to load the **calc** Python module and the **numpy** module.

The [calc module](http://localhost:8888/edit/Code/calc.py) is a non standard **Python** module that you can find in the [Code folder](http://localhost:8888/tree/Code)

**Execute** the following code to load those modules

In [ ]:
# We load numpy and calc
import numpy as np
import calc

# Check loaded module
try:
    print('calc version: ',calc.version)
except:
    print('Error loading the calc module')

## Obtaining the motor parameters

Before simulating the motor operation we will solve some of the motor equations because that can be usefull to deduce the motor parameters from the data provided by the motor manufacturer.

When the motor shaft is blocked (**stall** condition), angular speed and angular accelartion are zero.

$$\frac{k}{R}V -T_{stall}=0$$

Note that we can obtain the **current** from the **voltage** and **resistance** in the stall condition.

$$I_{stall}=\frac{V}{R}$$

The winding resistance $R$ can be obtained from the stall current and the operating voltage

$$R=\frac{V}{I_{stall}}$$

We can obtain the motor constant $k$ from the **stall current** and the **stall torque** of the motor.

$$k=\frac{T_{stall}R}{V}=\frac{T_{stall}}{I_{stall}}$$

A typical parameter given for the motor is the angular speed, in RPM, when it is not loaded. The angular speed $\omega$, in rad/s, can be obtained from the RPM value as:

$$\omega = \frac{2\pi RPM}{60}$$

We can deduce the friction coefficient $\mu$ from the unloaded speed of the motor. The motor, when is not loaded, has a zero external torque, so the maximum speed is reached when the generated torque equals the friction torque.

$$\frac{k}{R}(V-k \cdot \omega_{max}) = \mu \cdot \omega_{max}$$

So we obtain:

$$\mu = \frac{k}{\omega_{max} \cdot R}(V-k \cdot \omega_{max})$$

In good motors the friction is very small so it can be approximated to zero to ease the calculations. Note also that the inertial momentum **J** does not affect the maximum speed, only the time it takes to reach it.

The following code generates the motor internal parameters from the motor manufacturer information. 

**Execute** the code to obtain do the calculations.


In [ ]:
'''
Calculation of the parameters of a DC Motor
We use typical data obtained from a manufacturer
'''

# We will calculate the motor parameters from typical motor information

dcVoltage    = 12     # [V]
stallCurrent = 85     # [A]
stallTorque  = 0.5    # [N*m]
unloadedRPM  = 19300  # [min^-1]

# Calculations of constants of the motor

# Convert unloaded speed from RPM to rad/s
unloadedSpeed = 2.0*np.pi*unloadedRPM/60.0

# Calculate the winding resistance from the stall current
R = dcVoltage/stallCurrent

# Calculate the motor constant
k = stallTorque/stallCurrent

# Calculate the friction
mu = k*(dcVoltage-k*unloadedSpeed)/(R*unloadedSpeed)

# Show the results
calc.printVar("R",R,"Ohms")
calc.printVar("k",k,"N*m / A")
calc.printVar("mu",mu,"N*m / rad/s")

## Steady state motor operation

The motor will require some time to change from one speed to other, but, once reached a final speed, we will be at steady state and the inertial momentum will not affect the motor speed. So, in steady state:

$$\frac{k}{R}(V-k \cdot \omega) - \mu \cdot \omega -T_{L}=0$$

We can obtain the relationsheep between the load torque and the motor speed from the steady state conditions:

$$T_{L} = \frac{k}{R}(V-k \cdot \omega) - \mu \cdot \omega$$

This equation can also relate the angular speed with the other parameters

$$\omega = \frac{R}{k^2}(\frac{k \cdot V}{R}-T_{L})$$

We can also obtain the mechanical **power output** we can get from the motor

$$P_{M}=T_{L} \cdot \omega$$

The electrica **power input** required by the motor can also be calculated:

$$P_{E}=V \cdot I = V \cdot ( \frac{V-k \cdot \omega}{R} )$$

The **efficiency** of the motor can be calculated as the relation between the **output** mechanical power and the **input** electrical power

$$\eta = \frac{P_{M}}{P_{E}}$$

The following code calculates the **torque**, the **mechanical** and **electric** power and the **eficiency** of the motor as function of the motor speed. This data is shown in three graphs.

**Execute** the code to obtain the curves.


In [ ]:
# We define a vector for the speeds to calculate
vRPM=np.arange(0,unloadedRPM,unloadedRPM/200.0)

# We obtain the speed vector in rad/s
vRad=2.0*np.pi*vRPM/60.0

# We calculate the torque at each speed
vT=(k/R)*(dcVoltage-k*vRad)-mu*vRad

# And we show that on a graph
calc.plot11(vRPM,vT,"Torque vs speed","speed (RPM)","Torque (N*m)")

# We can also show the mechanical power and the input electrical power
vPout = vT*vRad
vPin  = dcVoltage*(dcVoltage-k*vRad)/R
calc.plot1n(vRPM,[vPout,vPin],"Power vs speed"
                 ,"speed (RPM)","Power (Watt)"
                 ,["Mechanical","Electrical"])

# We can also show the efficiency of the motor 
vEff=vPout/vPin
calc.plot11(vRPM,vEff,"Efficiency vs speed","speed (RPM)","Efficiency")
 

Note that the maximum efficiency is obtained just below the maximum speed of the motor.

## Dynamics of the motor

The motor dynamics when the speed or the torque changes can be obtained from the previously described motor equations:

$$T=\frac{k}{R}(V-k \cdot \omega) - \mu \cdot \omega -T_{L}$$

$$\omega=\frac{1}{J}\int T \cdot dt$$

We can rewrite the motor equations in a differential equation:

$$J \frac{d \omega}{dt}=\frac{k}{R}(V-k \cdot \omega) - \mu \cdot \omega -T_{L}$$

$$\frac{d \omega}{dt}+\frac{1}{J}(\frac{k^2}{R}+\mu)\omega+\frac{1}{J}(T_{L}-\frac{k \cdot V}{R})=0$$

Usually we can neglect the friction to ease the calculations

$$\frac{d \omega}{dt}+\frac{k^2}{J \cdot R}\omega+\frac{1}{J}(T_{L}-\frac{k \cdot V}{R})=0$$

Then we can solve the equation

$$\frac{d \omega}{\omega + \frac{R}{k^2}(T_{L}-\frac{k \cdot V}{R})}=-\frac{k^2}{J \cdot R}dt$$

$$\int\frac{d \omega}{\omega + \frac{R}{k^2}(T_{L}-\frac{k \cdot V}{R})}=-\frac{k^2}{J \cdot R}\int dt$$

$$ln (\omega + \frac{R}{k^2}(T_{L}-\frac{k \cdot V}{R}))=-\frac{k^2}{J \cdot R}t + C$$

$$\omega + \frac{R}{k^2}(T_{L}-\frac{k \cdot V}{R})=C' \cdot e^{-\frac{k^2}{J \cdot R}t}$$

$$\omega = \frac{R}{k^2}(\frac{k \cdot V}{R}-T_{L}) + C' \cdot e^{-\frac{k^2}{J \cdot R}t}$$

Note that the steady state (**SS**) angular speed is one element of the previous equation:

$$\omega = \omega_{ss} + C' \cdot e^{-\frac{k^2}{J \cdot R}t}$$

We can solve the C' constant if we know the angular speed at time t=0

$$\omega_{0} = \omega_{ss} + C' \cdot e^{-\frac{k^2}{J \cdot R}0}=\omega_{ss} + C'$$

So we can solve C'

$$C' = \omega_{0} - \omega_{ss}$$

So, in the end, the dinamics of the motor is:

$$\omega = \omega_{ss} + (\omega_{0} - \omega_{ss}) \cdot e^{-\frac{k^2}{J \cdot R}t}$$

Where, as we remember, the steady state angular speed is:

$$\omega_{ss} = \frac{R}{k^2}(\frac{k \cdot V}{R}-T_{L})$$

We can simulate the equations of our motor in a particular case. For instance, when we start the motor from a stopped position with no load attached

$$\omega = \omega_{ss}(1-e^{-\frac{k^2}{J \cdot R}t})$$

The steady state speed will be the maximum unloaded speed

The following code block obtains the analytical expression of the motor dynamics during the ramp-up. The dynamics are also obtained simulating the operation of the motor from its equations.

**Execute** the block to see the results.


In [ ]:
# Inertial momentum
J = 1e-3 #[kg*m^2]

# Time to solve for
tEnd =  20 #[s]
tStep = tEnd/1000.0
vTime = np.arange(0,tEnd,tStep)

# Analytical solution
tau = J*R/(k*k)
vAspeed = unloadedRPM*(1-np.exp(-vTime/tau))

# Show the time constant
calc.printVar("Time constant",tau,"s")

# Numerical solution ################################################

# We solve the system using the Euler method because it 
# is easier to understand and we only need qualitative results
# The result also is good enough compared with the analytical solution

# Start condition
speed = 0 #[rad/s]

# vector to store results
vSspeed = []

for time in vTime:
    # Acceleration
    alpha = (1/J)*((k/R)*(dcVoltage-k*speed)-mu*speed)
    # Speed
    speedNew = speed + alpha*tStep
    # Store result
    vSspeed.append(speed)
    # Update speed
    speed = speedNew
    
# Plot solutions
vSspeedRPM = np.array(vSspeed) * 60.0 / (2.0*np.pi)
calc.plot1n(vTime,[vAspeed,vSspeedRPM],"Unloaded motor ramp-up","Time (s)"
              ,"Speed (RPM)",["Analytical","Simulation"])

<BR><BR>

## Document information

Copyright © Vicente Jiménez (2018-2019)

Updated on 14/4/2019

This work is licensed under a [Creative Common Attribution-ShareAlike 4.0 International license](http://creativecommons.org/licenses/by-sa/4.0/). 

<img  src="images/cc_sa.png" width="200">